<a href="https://colab.research.google.com/github/thilankam/GraphNeuralNetwork_PyG/blob/main/Graph_NN_with_Node_set_U_and_V_for_Link_Prediction_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

1.13.1+cu116
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 59.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 22.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.2/280.2 KB 19.2 MB/s eta 0:00:00


In [2]:
import torch_geometric
from torch_geometric.data import Data

In [4]:
import torch
from torch_geometric.data import Data

# Define the number of nodes in each node set
num_u_nodes = 8
num_v_nodes = 235

# Create the node feature tensors for each node set
u_features = torch.randn(num_u_nodes, 10)
v_features = torch.randn(num_v_nodes, 10)

# Create the edge indices for the graph (no edges for this example)
edge_index = torch.tensor([], dtype=torch.long)

# Create a PyG Data object for the graph
data = Data(x=torch.cat([u_features, v_features], dim=0), edge_index=edge_index)

# Separate the node features into the U and V node sets
data.u_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
data.u_mask[:num_u_nodes] = 1
data.v_mask = ~data.u_mask

# Print some information about the graph
print(data)


Data(x=[243, 10], edge_index=[0], u_mask=[243], v_mask=[243])


In [5]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree

# Define the GNN model
class GNN(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(GNN, self).__init__(aggr='mean')  # "Mean" aggregation.
        self.lin = torch.nn.Linear(in_channels, out_channels)

    def forward(self, x, edge_index):
        # Add self-loops to the adjacency matrix.
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))

        # Compute the degree matrix.
        deg = degree(edge_index[0], x.size(0), dtype=x.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0

        # Apply graph convolution.
        x = self.lin(x)
        x = self.propagate(edge_index, x=x, norm=deg_inv_sqrt)
        return x

# Define the training and testing functions
def train(model, optimizer, train_loader):
    model.train()

    for data in train_loader:
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)[data.u_mask]
        loss = F.binary_cross_entropy_with_logits(out, data.y)
        loss.backward()
        optimizer.step()

def test(model, loader):
    model.eval()
    correct = 0

    for data in loader:
        with torch.no_grad():
            out = model(data.x, data.edge_index)[data.u_mask]
            pred = (out > 0).int()
            correct += int((pred == data.y).sum())

    acc = correct / len(loader.dataset)
    return acc

# Create a PyG Data object for the graph
data = ...

# Separate the node features into the U and V node sets
data.u_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
data.u_mask[:num_u_nodes] = 1
data.v_mask = ~data.u_mask

# Add ground-truth link labels for U-V node pairs
data.y = torch.zeros(data.num_nodes, dtype=torch.float)
u_idx, v_idx = data.edge_index
data.y[u_idx[data.u_mask], v_idx[data.v_mask]] = 1
data.y[v_idx[data.v_mask], u_idx[data.u_mask]] = 1

# Define the training and testing split indices
train_idx = torch.arange(num_u_nodes)
test_idx = torch.arange(num_u_nodes, data.num_nodes)

# Create the GNN model and optimizer
model = GNN(in_channels=10, out_channels=16)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Train and test the model
for epoch in range(100):
    train(model, optimizer, data[train_idx])
    acc = test(model, data[test_idx])
    print(f'Epoch {epoch+1}, Test accuracy: {acc:.4f}')


AttributeError: ignored